
# Acceptance Testing MRT Calculator



## Inputs

Update 5 - Parts Basket - Labour is read incorrectly when == 1
            Move upload and download file names to the beginning of the file

Update 4 - Inflation - Restructure Inflation Calculation

Update 3 - Maintenance - Fix calculation error

Update 2 - Services - Fix calculation error

Update 1 - Services - Move labour hours from parts file to baskets  

Service items & Tyres & Maintenance  

requiring multiple service items with labour hours  

Inflation - Price inflation of the components  

First Service - Run-in service    

Time or distance - Some components have a time limit.

Time based services - Annual service is distance not reached.

Items in service baskets

Import information from Excel  

Add cpk values

## Parameters for export

120 months - 6 month increments  
400,000 kms - 10000 km increments

## Convert the cost to a per month value


Export the table information in a format that matches the original


***Boilerplate imports***

In [1]:
import numpy as np
import pandas as pd

***Upload the cost data***

In [2]:


import pandas as pd
import ipywidgets as widgets
from IPython.display import display
import os

# List Excel files in current directory
files = [f for f in os.listdir('.') if f.endswith('.xlsx')]

# Dropdown widget
dropdown = widgets.Dropdown(options=files, description='Select file:')
display(dropdown)

def load_file(change):
    file_path = dropdown.value
    df = pd.read_excel(file_path)
    print(f"Loaded: {file_path}")
    display(df.head())

dropdown.observe(load_file, names='value')



download_filename = "F26460.xlsx"


Dropdown(description='Select file:', options=('AUMAN 25-460_F26ANI_CPK.xlsx', 'F26460.xlsx', 'Foton-Trucks-Sig…

In [7]:
# Load the corrected Excel file
xls = pd.ExcelFile(uploaded_file_path)

# Define Variables

# Load the sheets
df_info = pd.read_excel(xls, sheet_name="Information")
df_service = pd.read_excel(xls, sheet_name="Service Parts")
df_service_baskets = pd.read_excel(xls, sheet_name="Service Baskets")
df_tyre = pd.read_excel(xls, sheet_name="Tyre")
df_maintenance = pd.read_excel(xls, sheet_name="Maintenance")
df_out_warranty = pd.read_excel(xls, sheet_name="Out of Warranty")

# Extract configurations from the "Information" sheet
config = df_info.set_index("Parameter").squeeze().to_dict()
service_interval = int(config["service_interval"])
first_service = float(config["first_service"])
annual_service = bool(config["annual_service"])
inflation = float(config["inflation"])
labour_rate = float(config["labour_rate"])
table_id_code = str(config["table_id_code"])
full_table_name = str(config["full_table_name"])
vehicle_type = int(config["vehicle_type"])
export_limit = int(config["export_limit"])

# Extract Warranty information from the "Out of Warranty" sheet
warranty = df_out_warranty.set_index("Parameter").squeeze().to_dict()
std_war_km = int(warranty["Standard Warranty km"])
std_war_mnths = int(warranty["Standard Warranty months"])
std_war_prov = float(warranty["Standard Warranty provision ZAR"])
driv_war_km = int(warranty["Driveline Warranty km"])
driv_war_mnths = int(warranty["Driveline Warranty months"])
driv_war_prov = float(warranty["Driveline Warranty provision ZAR"])
out_war_prov = float(warranty["Out of Warranty provision ZAR"])
auth_lim_km = int(warranty["Authorisation limit km"])
over_lim_prov = float(warranty["Over limit provision ZAR"])

# Extract components and labour hours dynamically from the "Service Baskets" sheet
service_baskets = {}
services = df_service_baskets.columns[1:]  # Extracting service names (e.g., Service_1, Service_2, etc.)
components = df_service_baskets["Component"]

# Extract Labour Hours from the row labeled 'Labour Hours'
if "Labour Hours" in df_service_baskets["Component"].values:
    labour_hours_series = df_service_baskets.loc[df_service_baskets["Component"] == "Labour Hours"].iloc[0, 1:]
else:
    raise KeyError("Row 'Labour Hours' is missing in the 'Service Baskets' sheet")

# Convert Labour Hours into a dictionary for easier lookup
labour_hours_dict = labour_hours_series.to_dict()  # Service name -> Labour Hours


In [33]:
# Error checking - Labour Hours in basket

# Setting the "Component" column as the index
df_service_baskets.set_index('Component', inplace=True)

# Transposing the dataframe
df_transposed = df_service_baskets.transpose()

# Setting the "Labour Hours" column to float
df_transposed['Labour Hours'] = df_transposed['Labour Hours'].astype(float)

# Transposing the dataframe back to its original form
df_service_baskets = df_transposed.transpose()

df_service_baskets.reset_index(inplace=True)

In [34]:
for service in services:
    service_data = df_service_baskets[service].values  # Extract column data
    included_components = components[service_data == 1].tolist()  # Components included in the service    
    if "Labour Hours" in included_components:
        included_components.remove("Labour Hours")

    

    # Fetch Labour Hours for this service (if exists)
    labour_hours = labour_hours_dict.get(service, 1)  # Default to 1 if missing

    # Check if the service is a recurring service
    recurring_service = False
    if "Recurring Service" in components.values:
        recurring_service = service_data[components == "Recurring Service"][0] == "Yes"

    #recurring_interval = int(service_interval if recurring_service else 0)  # Use service interval if recurring
    recurring_interval = service_data[-2]

    # Build service basket dictionary
    service_baskets[service] = {
        "Components": included_components,
        "Recuring_Service": recurring_service,
        "Recuring_Interval": recurring_interval,
        "Labour Hours": labour_hours  # Fixed: Now correctly extracting from Service Baskets sheet
    }

In [35]:
# Extract tyre configuration from the "Tyre" sheet
df_tyre = df_tyre.set_index('Description')

try:
    steer_tyres, steer_interval, steer_price = df_tyre.loc["Steer Tyre"].to_list()
    drive_tyres, drive_interval, drive_price = df_tyre.loc["Drive Tyre"].to_list()
except KeyError:
    print('Tyre Information missing')

# Extract Maintenance information
df_maintenance = df_maintenance.set_index('Description')

maintenance_items = df_maintenance.index.to_list()

pd.ExcelFile.close(xls)

***Create the Cost Grids***

In [36]:
# Define grid parameters
months = np.arange(12, 121, 6)  # X-axis (6-month increments)
kms = np.arange(10000, export_limit + 1, 10000)  # Y-axis (10,000 km increments)

# Create an empty dataframe for cost grid - Services
cost_grid = pd.DataFrame(index=kms, columns=months)
cost_grid = cost_grid.infer_objects(copy=False).astype(float).fillna(0)

# Create an empty dataframe for cost grid - Tyres
cost_grid_tyres = pd.DataFrame(index=kms, columns=months)
cost_grid_tyres = cost_grid_tyres.infer_objects(copy=False).astype(float).fillna(0)

# Create an empty dataframe for cost grid - Maintenance
cost_grid_maintenance = pd.DataFrame(index=kms, columns=months)
cost_grid_maintenance = cost_grid_maintenance.infer_objects(copy=False).astype(float).fillna(0)

***Cost Calculation - Services***

In [37]:
# Extract the recurring intervals from the "Recurring Interval (km)" row
if "Recurring Interval (km)" in df_service_baskets["Component"].values:
    recurring_intervals_series = df_service_baskets.loc[df_service_baskets["Component"] == "Recurring Interval (km)"].iloc[0, 1:]
else:
    raise KeyError("Row 'Recurring Interval (km)' is missing in the 'Service Baskets' sheet.")

# Convert Recurring Intervals into a dictionary for easier lookup
recurring_intervals_dict = recurring_intervals_series.to_dict()  # Service name -> Recurring Interval (km)


In [38]:
# Generate a list if the services
services = []

# Initialize the interval counter
i = 1

# Use a while loop to iterate through the sequence
while (service_interval * i) <= export_limit:
    interval = service_interval * i 
    services.append(interval)
    i += 1


In [39]:
# Total cost per basket
service_name_cost = {}

for basket in service_baskets.keys():
    cost = 0
    # parts
    for item in service_baskets[basket]['Components']:
        
        cost += df_service.loc[df_service['Description'] == item, 'Price (ZAR)'].values[0]

    # labour
    cost += service_baskets[basket]['Labour Hours'] * labour_rate

    recurring_interval = service_baskets[basket]['Recuring_Interval']

    service_name_cost[basket] = [cost, recurring_interval]


In [40]:
# Generate a dict with the cost for each service
def basket_in_service(service, recurring_km):
    return service % recurring_km == 0

# Initialize the new dictionary
service_interval_cost = {}

# Calculate the cost of each service
for service in services:
    cost = 0   # reset cost counter for every service
    for basket in service_baskets.keys():
        price, recurring_km = service_name_cost[basket]       
        if (basket_in_service(service, recurring_km)):  # only add baskets that applies
            cost += price
    service_interval_cost[service] = cost


In [41]:
def calculate_total_service_cost(service_interval_cost, km_limit, month_limit, annual_service, service_interval, inflation):
    total_cost = 0
    
    # 1. Determine the effective annual distance (to calculate the year/inflation)
    annual_km_rate = (km_limit / month_limit) * 12 if month_limit > 0 else 0
    
    # 2. Iterate through all service events
    for service_km, base_cost in service_interval_cost.items():
        # Only include services up to the KM limit
        if service_km <= km_limit:
            
            # The service year is derived from the mileage and the annual rate
            # Year 1 = 0-12 months, Year 2 = 13-24 months, etc.
            service_year = max(1, int(service_km / annual_km_rate)) if annual_km_rate > 0 else 1 
            
            # The service event must also fall within the time limit
            if service_year * 12 <= month_limit:
                
                # Apply Inflation
                inflated_cost = base_cost * (1 + inflation)**(service_year - 1)
                total_cost += inflated_cost
                
    # 3. Handle Annual-only services (if annual_service is True)
    if annual_service:
        # If the km is not met, but time passes, an annual service is performed.
        # This is where your previous time-based logic should go.
        # This is complex and usually requires calculating the cost of a 'basic' annual service.
        # Assuming Service_1 (first key) is the base annual service cost
        # The true fix is in the main loop logic (Point 4)
        pass # Better handled in the main loop to avoid double counting

    return total_cost


# Service based on time or distance, WITH INFLATION APPLIED
for km in kms:
    for month in months:
        
        # Calculate the year/inflation factor
        year = month / 12
        inflation_factor = (1 + inflation)**(year - 1)
        
        # A. Determine which service schedule applies: Time or Distance
        # annual_km_travelled = (km / month) * 12
        
        # The correct approach is to run BOTH calculations and take the MAXIMUM cost, 
        # as the car should be serviced based on whichever comes first.
        
        # For simplicity, let's keep your original choice logic for now, 
        # but apply the inflation factor to the result:

        # NOTE: Your original logic for choosing is a bit non-standard;
        # A more robust model checks for 'Annual' if mileage isn't met, not 'service_interval'
        # if annual_km_travelled < service_interval:
        #     cost_grid.at[km, month] = service_cost_time(...) * inflation_factor
        # else:
        #     cost_grid.at[km, month] = service_cost_distance(...) * inflation_factor

        # ### REPLACEMENT LOGIC ###
        # **The cost must be calculated cumulatively, with inflation, up to the grid point.**
        
        annual_km_travelled = (km / month) * 12 if month > 0 else 0
        
        # 1. Calculate cost based on mileage (all services up to 'km' are included)
        mileage_cost = 0
        for service_km, base_cost in service_interval_cost.items():
            if service_km <= km:
                # Calculate the service year based on this vehicle's usage rate
                service_year = max(1, int(service_km / annual_km_travelled)) if annual_km_travelled > 0 else 1
                
                # Ensure the service falls within the total elapsed time (month)
                if service_year * 12 <= month:
                    mileage_cost += base_cost * (1 + inflation)**(service_year - 1)
                    
        # 2. Add time-based services (e.g., annual service) if annual_service is TRUE
        time_cost = 0
        if annual_service:
            # The annual service cost is typically the cost of the first, simplest service.
            # Assuming the cost of the first mileage service (key) is the annual service cost.
            annual_service_base_cost = service_interval_cost[services[0]]
            
            # Number of annual services performed due to TIME, not KM
            num_annual_services = month // 12
            
            for i in range(1, num_annual_services + 1):
                # The service year is 'i'
                # Check if a mileage service already covered this year's service (complex check, often simplified)
                # For simplicity, we just add the annual cost for each year, assuming they are mutually exclusive.
                time_cost += annual_service_base_cost * (1 + inflation)**(i - 1)


        # The final cost at (km, month) is the sum of mileage services and annual services
        # NOTE: This still risks double-counting. A robust model tracks which service was performed when.
        # For a simplified fixed-interval model, the first logic (calculating year from mileage) is often used.
        cost_grid.at[km, month] = mileage_cost # Sticking to the more robust mileage-based calculation

#### Service Cost Based on Distance (Accumulation) (Needs Inflation)

The service_cost_distance func currently sums up hte base ost of all services up to a given km point, but it does not apply inflation. Each service event should be inflated based on the year it occurs



* Current Logic


    $$TC = \sum_{k=1}^{serviceInter<=km} BaseCost_K$$



- Missing inflation, the cost of a service occuring in year Y should be Base Cost * (1+ inflation)^y-1


#### sERVICE Cost Based on Time (Incorrect logic)

- the service cost_time function is flawed because it attempts to use the mileage intervals (service_interval_cost.items()) to dtermine the number of annnual service

```{python}

def service_cost_time(service_interval_cost, month):
    cost = 0
    annual_services = month // 12
    
    for i, (key, value) in enumerate(service_interval_cost.items()):
        if (i < annual_services): # This is the issue
            cost += value
            
    return cost
```

- `service_interval_cost` keys are mileage(km) not time

- the `enumerate` index i is used as a proxy for the service number which is likely incorrect for services that occur annually (time-based)
 but are not direcrtly linked to the mileage intervals in your dictionary.

 - if annual_service is enabled, the model should apply a full-service cost (e.g., the cost of Service_1) for every 12 months, regardless of km


 #### Suggested Fixes (focus on Inflation)

 - replace two accumulation functions with a single, comprehensice function that handles both inflation and accumulation.

 - need auxilliary dfucntion to determine the year of a service event, based on the annual km rate (which is km/month * 12)

 ### fixed accumulatio function with inflation

 - this func calculates the total inflated cost up to a specific km and month point




 #### fixed main loop logic

 - logic to choose vetween distance and time is generally sound ,but we must incorporate inflation


 


***Cost Calculation - Tyres***

In [42]:
# Calculate the total cost of tyres based on intervals, WITH INFLATION
for km in kms:
    for month in months:
        
        total_cost = 0
        annual_km_rate = (km / month) * 12 if month > 0 else 0
        
        tyre_types = [
            (steer_interval, steer_tyres, steer_price),
            (drive_interval, drive_tyres, drive_price)
        ]
        
        for interval, num_tyres, base_price in tyre_types:
            if interval > 0:
                # Iterate through each replacement event
                for replacement_km in np.arange(interval, km + 1, interval):
                    
                    # Calculate the year the replacement occurs
                    if annual_km_rate > 0:
                        replacement_year = max(1, int(replacement_km / annual_km_rate)) 
                        
                        # Ensure the replacement is within the time limit
                        if replacement_year * 12 <= month:
                            # Apply Inflation
                            inflated_price = base_price * (1 + inflation)**(replacement_year - 1)
                            total_cost += num_tyres * inflated_price
                        
        cost_grid_tyres.at[km, month] = total_cost
# Convert to numeric
cost_grid_tyres = cost_grid_tyres.copy().astype(float)


# Tyre cost correction (Needs inflation)

- current tyre calculation is correct for the base cost but lacks inflation. total cost shoukd be the sum of each replacement even, with inflation applied to that events year

- Curent logic


$$TC = (\frac{km}{Inter_steer} \cdot Tyres \cdot Price_steer) + \dots$$

* fixed logic (with inflation)



***Cost Calculation - Maintenance***

### Calculate total base maintenace COST


In [43]:
# Assuming df_maintenance has 'Price (ZAR)' and 'Interval (months)' columns
annual_maintenance_base_cost = 0

# Calculate the base cost for maintenance occurring over the course of one year
for item in df_maintenance.index:
    try:
        # Assuming the base maintenance items are recurring annually (12 months)
        base_price = df_maintenance.loc[item, 'Price (ZAR)']
        interval_months = df_maintenance.loc[item, 'Interval (months)']
        
        # Calculate how many times this item occurs in 12 months
        occurrences_per_year = 12 / interval_months 
        annual_maintenance_base_cost += base_price * occurrences_per_year
    except KeyError as e:
        print(f"Warning: Missing column for maintenance calculation: {e}")
        
# For simplicity, if the maintenance items are just listed with a price, 
# you might just sum them up if they represent the total annual cost.
# Let's use the calculated 'annual_maintenance_base_cost'.



In [44]:
# Calculate the cost of maintenance based on information
def maintenance_cost(km, distance, price, hours, labour_rate, probability):
    probability = probability/100
    units = km//distance
    cost = 0
    cost += price * probability * units # parts
    cost += (hours * labour_rate) * probability * units # labour
    return cost


for km in kms:
    for month in months:
        cost = 0
        for item in maintenance_items:
            distance, price, hours, probability = df_maintenance.loc[item].to_list()            
            cost += maintenance_cost(km, distance, price, hours, labour_rate,probability)            
        
        cost_grid_maintenance.at[km, month] = cost

In [45]:
# --- Out of Warranty (OOW) and Over-Limit Provision ---
# NOTE: We assume 'out_war_prov' and 'over_lim_prov' are ZAR/km rates.
# Fixed ZAR provisions (std_war_prov, driv_war_prov) are NOT used here.

def out_of_warranty_cost(km, provision_rate):
    """Calculates accumulated cost based on a ZAR/km rate."""
    return km * provision_rate

for km in kms:
    for month in months:
        
        # 1. Base OOW Provision (ZAR/km)
        # This provision generally applies across the entire KM range.
        provision_rate = out_war_prov

        # 2. Over Limit Provision (ZAR/km)
        # This provision only applies to the distance past the limit.
        if km > auth_lim_km:
            # Add the over limit rate only for the distance exceeding the limit
            distance_over_limit = km - auth_lim_km
            
            # Accumulated cost for over-limit provision:
            over_limit_cost = distance_over_limit * over_lim_prov
            
            # Add this cost to the accumulated provision for this cell
            # The base OOW is still applied for the full KM range:
            oow_cost = out_of_warranty_cost(km, provision_rate)
            
            total_provision_cost = oow_cost + over_limit_cost
            
        else:
            # If within authorization limit, only apply the base OOW provision
            total_provision_cost = out_of_warranty_cost(km, provision_rate)
            
        # Add the total accumulated provision cost to the maintenance grid
        cost_grid_maintenance.at[km, month] += total_provision_cost
        


# Convert to numeric
cost_grid_maintenance = cost_grid_maintenance.copy().astype(float)

***Export values are a per month value***

The cell value is devided by the month value


In [46]:
# Normalize the values by dividing each by its corresponding month value
for month in months:
    cost_grid[month] = cost_grid[month] / month
    cost_grid_tyres[month] = cost_grid_tyres[month] / month
    cost_grid_maintenance[month] = cost_grid_maintenance[month] / month

***Inflation effect***

In [47]:
# Apply inflation to each month, assuming inflation compounds annually
inflated_cost_grid = cost_grid.copy() # Service Items
inflated_cost_grid_tyres = cost_grid_tyres.copy()  # Tyres
inflated_cost_grid_maintenance = cost_grid_maintenance.copy()  # Maintenance

def inflation_factor(inflation, month):
    """
    Calculates the average inflation factor for a cost accumulated over the given months.
    It sums the compound inflation factor for each year and divides by the number of years.
    """
    if inflation == 0:
        return 1.0
        
    years = month // 12
    if years == 0:
        return 1.0 # No inflation applied for less than one year

    # Sum the inflation factor for each year (year 0 is base year)
    sum_inflated_factors = 0
    for year in range(years):
        # (1 + inflation)**year gives the factor for the *end* of that year
        sum_inflated_factors += (1 + inflation)**year 

    # Return the average factor: Sum of factors / number of years
    return sum_inflated_factors / years

for month in months:  
     
    inflated_cost_grid[month] *= inflation_factor(inflation, month)  # Service
    inflated_cost_grid_tyres[month] *= inflation_factor(inflation, month)  # Tyres
    inflated_cost_grid_maintenance[month] *= inflation_factor(inflation, month)  # Maintenance



# Round all values to two decimal places
inflated_cost_grid = inflated_cost_grid.round(2)  # Service
inflated_cost_grid_tyres = inflated_cost_grid_tyres.round(2)  # Tyres
inflated_cost_grid_maintenance = inflated_cost_grid_maintenance.round(2)  # Maintenance

***cpk Sheet***

In [48]:
service_cpk_grid = pd.DataFrame(index=kms, columns=months)
tyre_cpk_grid = pd.DataFrame(index=kms, columns=months)
maintenance_cpk_grid = pd.DataFrame(index=kms, columns=months)

# --- Corrected CPK Calculation ---
for km in kms:
    if km == 0: continue # Avoid division by zero
    for month in months:
        # CPK for Service
        cpk_service = inflated_cost_grid.loc[km][month] / km
        service_cpk_grid.at[km, month] = cpk_service
        
        # CPK for Tyre
        cpk_tyre = inflated_cost_grid_tyres.loc[km][month] / km
        tyre_cpk_grid.at[km, month] = cpk_tyre

        # CPK for Maintenance
        cpk_maintenance = inflated_cost_grid_maintenance.loc[km][month] / km
        maintenance_cpk_grid.at[km, month] = cpk_maintenance

# Round all values to two decimal places
service_cpk_grid = service_cpk_grid.round(2)
tyre_cpk_grid = tyre_cpk_grid.round(2)
maintenance_cpk_grid = maintenance_cpk_grid.round(2)

In [49]:
service_cpk_grid = service_cpk_grid.reset_index()
tyre_cpk_grid = tyre_cpk_grid.reset_index()
maintenance_cpk_grid = maintenance_cpk_grid.reset_index()

cpk_grid = pd.concat([service_cpk_grid, tyre_cpk_grid, maintenance_cpk_grid], ignore_index=True)

## Format tables for export

***Combine information tables***

In [50]:
inflated_cost_grid = inflated_cost_grid.reset_index() # Service 208
inflated_cost_grid_tyres = inflated_cost_grid_tyres.reset_index() # Tyres 209
inflated_cost_grid_maintenance = inflated_cost_grid_maintenance.reset_index() # Maintenance 207

# Create information columns
inflated_cost_grid['A'] = ""
inflated_cost_grid['C'] = 208

inflated_cost_grid_tyres['A'] = ""
inflated_cost_grid_tyres['C'] = 209

inflated_cost_grid_maintenance['A'] = ""
inflated_cost_grid_maintenance['C'] = 207


output_grid = pd.concat([inflated_cost_grid, inflated_cost_grid_maintenance, inflated_cost_grid_tyres], ignore_index=True)

In [51]:
# Create information columns
output_grid['B'] = table_id_code
output_grid['E'] = full_table_name
output_grid['F'] = vehicle_type

In [52]:
# reorder columns
cols = [20, 22, 21, 0, 23, 24]
for i in range(1,20):
    cols.append(i)

output_grid = output_grid.iloc[:, cols]


# Export File



In [53]:
# Define the file name
mrt_export_sheet = "Export"
cpk_sheet = "cpk"

# Export the cost grid to an Excel file
with pd.ExcelWriter(download_filename) as writer:
    output_grid.to_excel(writer, sheet_name = mrt_export_sheet ,index=False)
    cpk_grid.to_excel(writer, sheet_name = cpk_sheet ,index=False)

print(f"Excel file '{mrt_export_sheet}' has been created successfully.")

Excel file 'Export' has been created successfully.
